In [ ]:
# design data model: 
"""
people: {
    id: string,
    name: string,
    age: int,
    gender: string,
    email: string,
    phone: string,
}
movies: {
    id: string,
    title: string,
    year: int,
    genre: string,
}

people -> movies: {
    person_id: string,
    movie_id: string,
    rating: int,
    timestamp: string,
}


"""

In [2]:
# init instance model

from neo4j import GraphDatabase

# Database connection configuration
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4jneo4j")

# Create driver instance
driver = GraphDatabase.driver(URI, auth=AUTH)

# Test connection
def test_connection():
    try:
        driver.verify_connectivity()
        print("Connection to Neo4j database successful!")
    except Exception as e:
        print(f"Failed to connect to Neo4j database: {e}")

test_connection()


Connection to Neo4j database successful!


In [3]:
# init instance model

cypher_query = """
CREATE (Apollo13:Movie {title: 'Apollo 13', tmdbID: 568, released: '1995-06-30', imdbRating: 7.6, genres: ['Drama', 'Adventure', 'IMAX']})
CREATE (TomH:Person {name: 'Tom Hanks', tmdbID: 31, born: '1956-07-09'})
CREATE (MegR:Person {name: 'Meg Ryan', tmdbID: 5344, born: '1961-11-19'})
CREATE (DannyD:Person {name: 'Danny DeVito', tmdbID: 518, born: '1944-11-17'})
CREATE (JackN:Person {name: 'Jack Nicholson', tmdbID: 514, born: '1937-04-22'})
CREATE (SleeplessInSeattle:Movie {title: 'Sleepless in Seattle', tmdbID: 858, released: '1993-06-25', imdbRating: 6.8, genres: ['Comedy', 'Drama', 'Romance']})
CREATE (Hoffa:Movie {title: 'Hoffa', tmdbID: 10410, released: '1992-12-25', imdbRating: 6.6, genres: ['Crime', 'Drama']})
"""

# Execute the query to import data
try:
    with driver.session() as session:
        result = session.run(cypher_query)
        print("Data imported successfully!")
except Exception as e:
    print(f"Failed to import data: {e}")


Data imported successfully!


In [16]:
rel_query = """
MATCH (TomH:Person {tmdbID: 31})
MATCH (MegR:Person {tmdbID: 5344})
MATCH (DannyD:Person {tmdbID: 518})
MATCH (JackN:Person {tmdbID: 514})

MATCH (Apollo13:Movie {tmdbID: 568})
MATCH (SleeplessInSeattle:Movie {tmdbID: 858})
MATCH (Hoffa:Movie {tmdbID: 10410})

MERGE (TomH)-[:ACTED_IN]->(Apollo13)
MERGE (TomH)-[:ACTED_IN]->(SleeplessInSeattle)
MERGE (MegR)-[:ACTED_IN]->(SleeplessInSeattle)
MERGE (DannyD)-[:ACTED_IN]->(Hoffa)
MERGE (DannyD)-[:DIRECTED]->(Hoffa)
MERGE (JackN)-[:ACTED_IN]->(Hoffa);
"""

with driver.session(database="neo4j") as session:
    session.run(rel_query)  # or session.execute_write(lambda tx: tx.run(rel_query))

In [17]:
# Query all ACTED_IN relationships

cypher_query = """
MATCH (person:Person)-[:ACTED_IN]->(movie:Movie)
RETURN person.name AS Actor, movie.title AS Movie
ORDER BY person.name, movie.title
"""

# Execute the query
try:
    with driver.session() as session:
        result = session.run(cypher_query)
        print("ACTED_IN Relationships:")
        print("-" * 40)
        for record in result:
            print(f"{record['Actor']} acted in '{record['Movie']}'")
except Exception as e:
    print(f"Failed to query data: {e}")


ACTED_IN Relationships:
----------------------------------------
Danny DeVito acted in 'Hoffa'
Jack Nicholson acted in 'Hoffa'
Meg Ryan acted in 'Sleepless in Seattle'
Tom Hanks acted in 'Apollo 13'
Tom Hanks acted in 'Sleepless in Seattle'


In [12]:
# Query all nodes in the database

cypher_query = """
MATCH (n)
RETURN n
"""

# Execute the query
try:
    with driver.session() as session:
        result = session.run(cypher_query)
        print("All Nodes in Database:")
        print("-" * 40)
        for record in result:
            node = record['n']
            labels = ':'.join(node.labels)
            properties = dict(node)
            print(f"Labels: {labels}")
            print(f"Properties: {properties}")
            print("-" * 20)
except Exception as e:
    print(f"Failed to query nodes: {e}")


All Nodes in Database:
----------------------------------------
Labels: Movie
Properties: {'tmdbID': 568, 'genres': ['Drama', 'Adventure', 'IMAX'], 'imdbRating': 7.6, 'title': 'Apollo 13', 'released': '1995-06-30'}
--------------------
Labels: Person
Properties: {'tmdbID': 31, 'born': '1956-07-09', 'name': 'Tom Hanks'}
--------------------
Labels: Person
Properties: {'tmdbID': 5344, 'born': '1961-11-19', 'name': 'Meg Ryan'}
--------------------
Labels: Person
Properties: {'tmdbID': 518, 'born': '1944-11-17', 'name': 'Danny DeVito'}
--------------------
Labels: Person
Properties: {'tmdbID': 514, 'born': '1937-04-22', 'name': 'Jack Nicholson'}
--------------------
Labels: Movie
Properties: {'tmdbID': 858, 'genres': ['Comedy', 'Drama', 'Romance'], 'imdbRating': 6.8, 'title': 'Sleepless in Seattle', 'released': '1993-06-25'}
--------------------
Labels: Movie
Properties: {'tmdbID': 10410, 'genres': ['Crime', 'Drama'], 'imdbRating': 6.6, 'title': 'Hoffa', 'released': '1992-12-25'}
---------

In [13]:
# query all edges in the database

cypher_query = """
MATCH ()-[r]->()
RETURN r
"""

# Execute the query
try:
    with driver.session() as session:
        result = session.run(cypher_query)
        print("All Edges in Database:")
        print("-" * 40)
        for record in result:
            edge = record['r']
            print(f"Edge: {edge}")
            print("-" * 20)
except Exception as e:
    print(f"Failed to query edges: {e}")

All Edges in Database:
----------------------------------------
Edge: <Relationship element_id='5:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:1152921504606846977' nodes=(<Node element_id='4:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:1' labels=frozenset() properties={}>, <Node element_id='4:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:0' labels=frozenset() properties={}>) type='ACTED_IN' properties={}>
--------------------
Edge: <Relationship element_id='5:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:1152922604118474753' nodes=(<Node element_id='4:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:1' labels=frozenset() properties={}>, <Node element_id='4:acc9f7b3-cc38-4a48-b9c6-f3c424f88e4a:0' labels=frozenset() properties={}>) type='DIRECTED' properties={}>
--------------------
